In [1]:
from google.cloud import storage, aiplatform

In [2]:
ARTIFACT_FILENAME = "model.pkl"
BUCKET = "iris-classification"
ARTIFACT_URI = f"gs://{BUCKET}"
MODEL = "iris"
ENDPOINT = "iris"

# the serving container image
# https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
DEPLOY_IMAGE = "europe-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest"

PROJECT = "cloud4us-gcp-3fbr5mcqluov7itt4"
LOCATION = "europe-west1"

# create Cloud Storage bucket

In [3]:
client = storage.Client()
bucket = client.bucket(BUCKET)
if not bucket.exists():
    new_bucket = client.create_bucket(bucket, location="europe-west1")

# copy local .pkl file to Cloud Storage

In [4]:
!gsutil cp {ARTIFACT_FILENAME} {ARTIFACT_URI}

Copying file://model.pkl [Content-Type=application/octet-stream]...
/ [1 files][174.2 KiB/174.2 KiB]                                                
Operation completed over 1 objects/174.2 KiB.                                    


# init aiplatform

In [5]:
aiplatform.init(project=PROJECT, location=LOCATION)

# upload model to Vertex AI

In [6]:
%%time

model = aiplatform.Model.upload(
    display_name=MODEL,
    artifact_uri=ARTIFACT_URI,
    serving_container_image_uri=DEPLOY_IMAGE
)

Creating Model
Create Model backing LRO: projects/704710201124/locations/europe-west1/models/4002063995430240256/operations/3333931461160992768
Model created. Resource name: projects/704710201124/locations/europe-west1/models/4002063995430240256@1
To use this Model in another session:
model = aiplatform.Model('projects/704710201124/locations/europe-west1/models/4002063995430240256@1')
CPU times: user 73.6 ms, sys: 18.6 ms, total: 92.2 ms
Wall time: 3.43 s


# deploy model to endpoint

In [7]:
%%time

endpoint=model.deploy(deployed_model_display_name=ENDPOINT)

Creating Endpoint
Create Endpoint backing LRO: projects/704710201124/locations/europe-west1/endpoints/9209949198902886400/operations/4679381849837928448
Endpoint created. Resource name: projects/704710201124/locations/europe-west1/endpoints/9209949198902886400
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/704710201124/locations/europe-west1/endpoints/9209949198902886400')
Deploying model to Endpoint : projects/704710201124/locations/europe-west1/endpoints/9209949198902886400
Using default machine_type: n1-standard-2
Deploy Endpoint model backing LRO: projects/704710201124/locations/europe-west1/endpoints/9209949198902886400/operations/560839990607609856
Endpoint model deployed. Resource name: projects/704710201124/locations/europe-west1/endpoints/9209949198902886400
CPU times: user 112 ms, sys: 20.1 ms, total: 132 ms
Wall time: 47.9 s


# 🧹 [optional] clean up

In [8]:
endpoint.delete(force=True)  # force=True to undeploy all models before deleting endpoint

Undeploying Endpoint model: projects/704710201124/locations/europe-west1/endpoints/4675950274047639552
Undeploy Endpoint model backing LRO: projects/704710201124/locations/europe-west1/endpoints/4675950274047639552/operations/2119085461677801472
Endpoint model undeployed. Resource name: projects/704710201124/locations/europe-west1/endpoints/4675950274047639552
Deleting Endpoint : projects/704710201124/locations/europe-west1/endpoints/4675950274047639552
Delete Endpoint  backing LRO: projects/704710201124/locations/europe-west1/operations/6099141632366477312
Endpoint deleted. . Resource name: projects/704710201124/locations/europe-west1/endpoints/4675950274047639552


In [9]:
model.delete()

Deleting Model : projects/704710201124/locations/europe-west1/models/4639323342703165440
Delete Model  backing LRO: projects/704710201124/locations/europe-west1/operations/6613677889793556480
Model deleted. . Resource name: projects/704710201124/locations/europe-west1/models/4639323342703165440


In [10]:
# Delete Cloud Storage bucket
!gcloud storage rm --recursive {ARTIFACT_URI}

Removing objects:
Removing gs://iris-classification/model.pkl#1666532621090516...                
  Completed 1/1                                                                
Removing Buckets:
Removing gs://iris-classification/...                                          
  Completed 1/1                                                                
